In [1]:
import math

import matplotlib.pyplot as plt
import numpy as np
import torch
import torchdyn

from torchdyn.datasets import generate_moons
from utils import sample_8gaussians,sample_moons

In [7]:
generate_moons(10, noise=0.2)

(tensor([[ 1.0469,  0.0469],
         [ 0.8699,  0.8699],
         [ 0.0194,  1.0194],
         [-0.6167,  0.7975],
         [-0.9594,  0.0406],
         [ 0.0766,  0.5766],
         [ 0.3884, -0.1116],
         [ 1.0736, -0.4264],
         [ 1.8040, -0.1102],
         [ 2.1741,  0.6741]]),
 tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1]))

In [5]:
batch_size = 32
x0 = sample_8gaussians(batch_size)
x1 = sample_moons(batch_size)